In [1]:
import os
import random
import dspy
import logging

import dspy.evaluate

logger = logging.getLogger(__name__)



/home/vikyw/.cache/pypoetry/virtualenvs/src-QP2L3v8R-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
from dspy import Example
from csv import DictReader


def load_training_data_as_example(name: str) -> list[Example]:
    output = []
    with open(f"src/data/{name}") as f:
        file = DictReader(f)

        for row in file:
            output.append(Example(row).with_inputs("vn"))

    return output

In [36]:
trainset = load_training_data_as_example(name="20240827-training.csv")
trainset

[Example({'vn': 'Vui lòng cấp giấy chứng nhận số tiền còn lại.', 'ko': '잔액증명서를 발급해주세요'}) (input_keys={'vn'}),
 Example({'vn': 'Hãy làm thiệp trả trước đi ạ', 'ko': '선불카드 만들어주세요'}) (input_keys={'vn'}),
 Example({'vn': 'Tạo tài khoản cho mình đi', 'ko': '계좌 만들어주세요'}) (input_keys={'vn'}),
 Example({'vn': 'Gửi tiền cho mình đi', 'ko': '송금해주세요'}) (input_keys={'vn'}),
 Example({'vn': 'Vui lòng cấp lại thẻ bảo mật.', 'ko': '보안카드 재발급해주세요'}) (input_keys={'vn'}),
 Example({'vn': 'Tôi muốn khai báo thẻ bị mất.', 'ko': '분실된 카드를 신고하고 싶어요.'}) (input_keys={'vn'}),
 Example({'vn': 'Tôi muốn kiểm tra số dư tài khoản.', 'ko': '계좌 잔액을 확인하고 싶습니다'}) (input_keys={'vn'}),
 Example({'vn': 'Tôi muốn đăng ký ngân hàng internet', 'ko': '인터넷 뱅킹을 등록하고 싶어요'}) (input_keys={'vn'}),
 Example({'vn': 'Hãy đổi tiền cho tôi.', 'ko': '환전해 주세요'}) (input_keys={'vn'}),
 Example({'vn': 'Tôi muốn đăng ký thẻ kiểm tra', 'ko': '체크카드를 신청하고 싶어요'}) (input_keys={'vn'}),
 Example({'vn': 'Tôi muốn tăng giới hạn thẻ tín dụng.', 'ko': '신

In [37]:
from dspy.evaluate import Evaluate

llm = dspy.OpenAI(
    model="meta-llama/llama-3.1-70b-instruct",
    base_url=os.getenv("OPENROUTER_BASE_URL"),
    api_key=os.getenv("OPENROUTER_API_KEY"),
    temperature=0.2,
)

dspy.configure(lm=llm)

score_accuracy = dspy.evaluate.metrics.answer_passage_match
evaluator = Evaluate(devset=trainset, display_progress=True)

In [39]:
class Translation(dspy.Signature):
    """Translate a text from Vietnamese to Korean"""
    vn = dspy.InputField(desc="Vietnamese text to translate")
    ko = dspy.OutputField(desc="Korean translation")

class ScoNeCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(Translation)

    def forward(self, input):
        return self.generate_answer(input=input.vn)


In [40]:
cot_zeroshot = ScoNeCoT()
evaluator(cot_zeroshot, metric=score_accuracy,display_progress=True)

  0%|          | 0/11 [00:00<?, ?it/s]

2024-08-26T17:05:20.408933Z [error    ] Error for example in dev set: 		 ScoNeCoT.forward() got an unexpected keyword argument 'vn' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 1  (0.0):   9%|▉         | 1/11 [00:00<00:00, 285.48it/s]

2024-08-26T17:05:20.411170Z [error    ] Error for example in dev set: 		 ScoNeCoT.forward() got an unexpected keyword argument 'vn' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 2  (0.0):  18%|█▊        | 2/11 [00:00<00:00, 334.83it/s]

2024-08-26T17:05:20.413421Z [error    ] Error for example in dev set: 		 ScoNeCoT.forward() got an unexpected keyword argument 'vn' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 3  (0.0):  27%|██▋       | 3/11 [00:00<00:00, 382.10it/s]

2024-08-26T17:05:20.415657Z [error    ] Error for example in dev set: 		 ScoNeCoT.forward() got an unexpected keyword argument 'vn' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 4  (0.0):  27%|██▋       | 3/11 [00:00<00:00, 358.27it/s]

TypeError: ScoNeCoT.forward() got an unexpected keyword argument 'vn'